In [1]:
pip --version

pip 23.0.1 from C:\Users\innovation_lab\Anaconda3\lib\site-packages\pip (python 3.7)

Note: you may need to restart the kernel to use updated packages.


In [2]:
from platform import python_version
print(python_version())

3.7.3


In [1]:
sampleText1 = "Welcome to BMW Metaverse world, how can I help today?"
# need to find "help"
sampleText2 = "The Avatar opens the door for you"

sampleLongText = "BMW regularly reviews and evaluates\
its product pricing to flexibly respond to changes in raw materials and vehicle manufacturing costs"
# review/evaluate.   flexiable price 

# NLTK-Score/WordType

In [39]:
import numpy as np
import pandas as pd 
import os 
import json
import string

In [3]:
from rake_nltk import Rake
from spacy import displacy
import nltk
import operator

Using TensorFlow backend.


In [4]:
from numba import cuda
import numpy as np
import pandas as pd
import spacy

In [5]:
nltk.download(['stopwords','punkt',"names","stopwords","state_union","twitter_samples","movie_reviews","averaged_perceptron_tagger","vader_lexicon","punkt"])

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\innovation_lab\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\innovation_lab\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package names to
[nltk_data]     C:\Users\innovation_lab\AppData\Roaming\nltk_data...
[nltk_data]   Package names is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\innovation_lab\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package state_union to
[nltk_data]     C:\Users\innovation_lab\AppData\Roaming\nltk_data...
[nltk_data]   Package state_union is already up-to-date!
[nltk_data] Downloading package twitter_samples to
[nltk_data]     C:\Users\innovation_lab\AppData\Roaming\nltk_data...
[nltk_data]   Package twitter_samples is alread

True

In [6]:
spacy.cli.download("en_core_web_sm")

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [31]:
from nltk.stem.wordnet import WordNetLemmatizer

## Word Score

In [7]:
# string to ignore
r = Rake(punctuations = [')','(',',',':','),',').','.','?','~','!'])

In [8]:
typesNeed = ["VBP", "VBN", "VBG", "VBD", "VB", "NN","NNS", "NNP", "NNPS"]

In [9]:
# train the model
r.extract_keywords_from_text(sampleText1)

In [10]:
# get score of each word, sorted already
# sampleText1 = "Welcome to BMW Metaverse world, how can I help today?"
phrase_df = pd.DataFrame(r.get_ranked_phrases_with_scores(), columns = ['score','phrase'])
phrase_df.head(5)

,score,phrase
0,9.0,bmw metaverse world
1,4.0,help today
2,1.0,welcome


## Word Type

In [11]:
# demo token/postTag
# sampleText1 = "Welcome to BMW Metaverse dealer, how can I help today?"
storeTypes = {}
s = nltk.word_tokenize(sampleText1)
wordsType = nltk.pos_tag(s)
print(wordsType)
print("\n")
temPhrase = []
temTypes = []
for w in wordsType:
    if w[1] not in typesNeed:
        if not temPhrase or not temTypes:
            continue
        else:
            phrase = " ".join(temPhrase).lower()
            storeTypes[phrase] = temTypes.copy()
            temPhrase.clear()
            temTypes.clear()
    else:
        temPhrase.append(w[0])
        temTypes.append(w[1])
print(storeTypes)

[('Welcome', 'VB'), ('to', 'TO'), ('BMW', 'NNP'), ('Metaverse', 'NNP'), ('world', 'NN'), (',', ','), ('how', 'WRB'), ('can', 'MD'), ('I', 'PRP'), ('help', 'VB'), ('today', 'NN'), ('?', '.')]


{'welcome': ['VB'], 'bmw metaverse world': ['NNP', 'NNP', 'NN'], 'help today': ['VB', 'NN']}


## Configuration of Pegasus

In [13]:
# import and configure Pegasus Model
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
import torch

In [14]:
# Create Tokenizer
# can try different model 
tokenizer = PegasusTokenizer.from_pretrained("google/pegasus-xsum")

In [15]:
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch_device

'cpu'

In [16]:
# create model
model = PegasusForConditionalGeneration.from_pretrained("google/pegasus-xsum")

## Perform Abstractive Summarization

In [18]:
# create tokens--number representation of our text
#tokens = tokenizer(paragraph1, truncation=True, padding="longest", return_tensors="pt")

In [19]:
nlp = spacy.load("en_core_web_sm")

In [20]:
# sampleText2 = "The Avatar opens the door for you"
displacy.render(nlp(sampleText2), style = "dep", jupyter = True)

## Noun Verb Noun

In [21]:
def nvn_split_punc(text):
    # Storing the sets of punctuation in variable result
    result = string.punctuation

    for i in text:
        if i in result:
            test_str=text.replace(i,"*"+i+"*")
    res=text.split("*")
    return res

In [22]:
# nsubj: object of the verb 
# nsubjpass: passive nominal subject
# dobj: direct object 
# iobj: indirect object 
def nvn(text):
    texts = nvn_split_punc(text)
    
    sents = list()
    
    for t in texts:
        doc = nlp(t)
        sent = []

        for token in doc:
            # if the token is a verb
            if (token.pos_=='VERB'):
                phrase =''
                left = ''
                right = ''
                # only extract noun or pronoun subjects
                for sub_tok in token.lefts:
                    if (sub_tok.dep_ in ['nsubj','nsubjpass']) and (sub_tok.pos_ in ['NOUN','PROPN','PRON']):
                        # add subject to the phrase
                        phrase += sub_tok.text
                        # save the root of the verb in phrase
                        phrase += ' '+token.lemma_ 
                        # check for noun or pronoun direct objects
                        for sub_tok in token.rights:
                            # save the object in the phrase
                            if (sub_tok.dep_ in ['dobj', 'iobj','pobj']) and (sub_tok.pos_ in ['NOUN','PROPN']):
                                phrase += ' '+sub_tok.text
                                sent.append(phrase)

                        if len(sent)>0:  
                            print(sent)
                            sents.append(sent)
    return sents


In [23]:
# sampleText2 = "The Avatar opens the door for you"
def get_df_nvn(text):
    output = nvn(text)
    print(output)

    # iterating over all the outputs from the sentence
    res_dict = list()
    for sent in output:
        for phrase in sent:
            n1, v, n2 = phrase.split()[:1], phrase.split()[1], phrase.split()[2:]
            # append to list, along with the sentence
            dis_dict = {'Sent':phrase,'Noun1':n1,'Verb':v,'Noun2':n2}
            res_dict.append(dis_dict)

    df_nvn = pd.DataFrame(res_dict)
    return df_nvn
get_df_nvn(sampleText2)
#******** what if there is multiple nsubj or obj


['Avatar open door']
[['Avatar open door']]


,Noun1,Noun2,Sent,Verb
0,[Avatar],[door],Avatar open door,open


In [24]:
# sampleText2 = "The Avatar opens the door for you"
displacy.render(nlp(sampleText1), style = "dep", jupyter = True)

In [25]:
sampleText3 = "Never had she read such a sad book as The Coldest Tear."

get_df_nvn(sampleText3)

['she read book']
[['she read book']]


,Noun1,Noun2,Sent,Verb
0,[she],[book],she read book,read


# Command Extraction

## Adjective Noun Structure

In [80]:
# nsubj: object of the verb 
# nsubjpass: passive nominal subject
# dobj: direct object 
# iobj: indirect object 
def nvn_an(text):
    texts = nvn_split_punc(text)
    
    # 
    sents = []
    for t in texts:
        doc = nlp(t)
        #displacy.render(doc, style = "dep", jupyter = True)
        # I open the back door:-> key: door value:{key(nvn: [I open door], adj-n: [back door])}
        sent = dict()
        pat = list()
        
        for token in doc:
            noun_verb = ''
            phrase_adj = ''
            
            one_sent = {
                "verb":noun_verb,
                "adj":[]
            }
            
            # if the token is a verb
            if (token.pos_=='NOUN')and(token.dep_ in ['dobj','pobj','nsubj','nsubjpass']):
                # iterate over the children nodes
                for subtoken in token.children:
                    # if word is an adjective or has a compound dependency
                    if (subtoken.pos_ == 'ADJ') or (subtoken.dep_ == 'compound'):
                        phrase_adj += subtoken.text
                if token.head.pos_ is not None and token.head.pos_ == "VERB":
                    noun_verb = token.head.text
                    
                if len(phrase_adj) > 0:
                    pat.append(phrase_adj)
                            
            if len(noun_verb)>0:
                one_sent["verb"]=noun_verb
            if len(phrase_adj)>0:
                one_sent["adj"] = pat.copy()
            if len(noun_verb)>0 or len(pat)>0:
                sent[token.text]=one_sent
            
            pat.clear()

    return sent


In [81]:
nvn_an("open the front door")

{'door': {'verb': 'open', 'adj': ['front']}}

In [82]:
nvn_an("change the interior color to blue")

{'color': {'verb': 'change', 'adj': ['interior']}}

In [83]:
fileName = "cmdDict.json"
f = open(fileName)
data = json.load(f)
autoCmd = data["auto"]["nouns"]
autoCmd

{'Position': {'move': ['which directions do you want to move your car?',
   'How far do you want your car to go?']},
 'color': {'change': ['what color do you want to change your car?',
   'which part of car do you want to change the color?']},
 'lights': {'turn on': ['which lights do you want to turn on?'],
  'turn off': ['which lights do you want to turn off?']},
 'door': {'open': ['which door do you want to open?'],
  'close': ['which door do you want to close']},
 'wheel': ['what types of wheels do you want to change or replace?']}

In [84]:
def cmdInference(cmd,df):
    if cmd is None:
        return None
    res = nvn_an(cmd)
    print(res)
    # all decorative nouns
    for n in res.keys():
        # if the result dictionary has verb in local dataset
        if n in autoCmd:
            if len(res[n]["verb"])==0:
                raise Exception("Invalid Command without verb")
            else:
                queryVerb = res[n]["verb"]
                queryAdj = res[n]["adj"]
                if queryVerb in autoCmd[n]:
                    if len(queryAdj)==0:
                        for question in autoCmd[n][queryVerb]:
                            print(question)
                    else:
                        print("action")
                        
        else:
            print("invalid Command")
        

In [89]:
#"change the interior color to blue"
cmdInference("change the color",autoCmd)

{'color': {'verb': 'change', 'adj': []}}
what color do you want to change your car?
which part of car do you want to change the color?


In [90]:
cmdInference("open the door",autoCmd)

{'door': {'verb': 'open', 'adj': []}}
which door do you want to open?


In [87]:
cmdInference("open the back door",autoCmd)

{'door': {'verb': 'open', 'adj': ['back']}}
action
